In [159]:
import scipy.io
import mat73
import pandas as pd
import numpy as np
from numpy import array
from numpy.random import uniform
from numpy import hstack
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, LeakyReLU, BatchNormalization
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.svm import SVC as svm
from sklearn.linear_model import LogisticRegression as lg
from sklearn.metrics import confusion_matrix,accuracy_score,balanced_accuracy_score,f1_score
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.exceptions import ConvergenceWarning
from warnings import simplefilter,filterwarnings

# ignore all future warnings1
simplefilter(action='ignore', category=FutureWarning)



In [160]:
import os

DATA_DIR = ''
if 'google.colab' not in str(get_ipython()):
    if "anuja" in os.environ.get('USER'):
        DATA_DIR = 'data/'
    elif 'ubuntu' in os.environ.get('USER'):
        DATA_DIR = '/home/ubuntu/Martyna/repo/AI4Health/DATAfoof/'
    

## EEG data and foof:

In [161]:
# ## data loading
# data = scipy.io.loadmat(DATA_DIR+'x.mat')  
# foof = pd.read_csv(DATA_DIR+"sensor_intercept_slope.csv")
# foof

In [162]:
# # flattening
# df = pd.DataFrame(data['x'].reshape((data['x'].shape[0], -1)))
# df = np.array(df)
# df = df.reshape(data['x'].shape)
# df.shape

In [163]:
# df = pd.DataFrame(df.reshape((df.shape[0], -1)))
# df['IDs'] = foof['IDs']
# df

## Behavioral Data:


In [164]:
# behaviour_data = pd.read_csv(DATA_DIR+'AllData.csv')
# behaviour_data = behaviour_data.rename(columns = {'EID': 'IDs'}, inplace = False)
# behaviour_data.columns

In [165]:
# beh = pd.read_csv(DATA_DIR+"behaviorals.csv")
# add_features = beh[['Sex', 'Age','IDs']]
# add_features.shape

In [166]:
# behaviour_data = behaviour_data[['IDs', 'SRS_SCI_T', 'SRS_RRB_T', 'SWAN_IN_Avg', 'SWAN_HY_Avg',
#        'SWAN_Avg', 'SCARED_P_GD', 'WISC_WMI_Sum',
#        'WISC_VCI_Sum']]
# behaviour_data.shape

In [167]:
# def missing_values_table(df):
#     mis_val = df.isnull().sum()
#     mis_val_percent = 100 * df.isnull().sum() / len(df)
#     mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
#     mis_val_table_ren_columns = mis_val_table.rename(
#     columns = {0 : 'Missing Values', 1 : '% of Total Values'})
#     mis_val_table_ren_columns = mis_val_table_ren_columns[
#         mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
#     '% of Total Values', ascending=False).round(1)
#     print ("Your selected dataframe has " + str(df.shape[1]) + " columns (with column 'IDs').\n"      
#         "There are " + str(mis_val_table_ren_columns.shape[0]) +
#             " columns that have missing values.")
#     return mis_val_table_ren_columns

# missing_values_table(behaviour_data)

## Merge EEG and foof data together:

In [168]:
# data = pd.merge(df, foof, on='IDs', how='inner')
# data.shape

## and then with the behaviorals

In [169]:
# df = pd.merge(data, behaviour_data, on='IDs', how='inner')
# df.shape

In [170]:
# #add age and gender
# df = pd.merge(df, add_features, on='IDs', how='inner')
# df.shape

## features and labels preparation:

In [171]:
# #removing NaNs
# df = df.dropna()
# df.shape

In [172]:
# labels_list = [ 'SRS_SCI_T', 'SRS_RRB_T', 'SWAN_IN_Avg', 'SWAN_HY_Avg',
#    'SWAN_Avg', 'SCARED_P_GD', 'WISC_WMI_Sum',
#    'WISC_VCI_Sum']

# X = df[df.columns.difference(['IDs']+labels_list)]
# print(X.shape)

In [173]:
# Y = df[labels_list]
# print(Y.shape)a

## Split and PCA



In [174]:
# xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.25, shuffle=True)

path = os.path.join(DATA_DIR, 'multilabel_regression_sensor_level')

# #saving
# if not(os.path.exists(path)):
#     os.mkdir(path)
# np.save(os.path.join(path, 'xtrain.npy'), xtrain)
# np.save(os.path.join(path, 'xtest.npy'), xtest)
# np.save(os.path.join(path, 'ytrain.npy'), ytrain)
# np.save(os.path.join(path, 'ytest.npy'), ytest)


#loading
xtrain = np.load(os.path.join(path, 'xtrain.npy'))
xtest = np.load(os.path.join(path, 'xtest.npy'))
ytrain = np.load(os.path.join(path, 'ytrain.npy'))
ytest = np.load(os.path.join(path, 'ytest.npy'))

In [175]:
age_gender = True
if age_gender:
    train_age_gender = xtrain[:,-2:]
    test_age_gender = xtest[:,-2:]
    xtrain = xtrain[:,:-2]
    xtest = xtest[:,:-2]

In [176]:
# scaling x
norm = preprocessing.MinMaxScaler().fit(xtrain)

# transform training data
xtrain = norm.transform(xtrain)
xtest = norm.transform(xtest)
print(xtrain.shape, xtest.shape)

print('Applying PCA...')
pca = PCA(.95) # 95% variance retained
pca.fit(xtrain)

# transform data
xtrain = pca.transform(xtrain)
xtest = pca.transform(xtest)
print(xtrain.shape, xtest.shape)

(804, 8505) (268, 8505)
Applying PCA...
(804, 194) (268, 194)


In [177]:
if age_gender:
    xtrain = np.concatenate([xtrain, train_age_gender], axis = 1)
    xtest = np.concatenate([xtest, test_age_gender], axis = 1)

In [178]:
# scaling y
min_max_scaler = preprocessing.MinMaxScaler().fit(ytrain)
ytrain = min_max_scaler.transform(ytrain)
ytest = min_max_scaler.transform(ytest)

print(ytrain.shape, ytest.shape)

(804, 8) (268, 8)


In [179]:
ytrain.mean(axis=0)

array([0.41457067, 0.3350594 , 0.61818219, 0.54341715, 0.58079966,
       0.25725539, 0.44997668, 0.49228124])

## MODELLING PART

### dummy regressors (to obtain the random baseline):

In [180]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, median_absolute_error
from sklearn.dummy import DummyRegressor


lm_dummy_mean = DummyRegressor(strategy = 'mean').fit(xtrain, ytrain)
lm_dummy_median = DummyRegressor(strategy = 'median').fit(xtrain, ytrain)
ypred_dummy_mean = lm_dummy_mean.predict(xtest)
ypred_dummy_median = lm_dummy_median.predict(xtest)


print("Mean squared error (dummy): {:.2f}".format(mean_squared_error(ytest,
																	ypred_dummy_mean)))

print("Median absolute error (dummy): {:.2f}".format(median_absolute_error(ytest,
																	ypred_dummy_median)))

print("r2_score (dummy mean): {:.2f}".format(r2_score(ytest, ypred_dummy_mean)))
print("r2_score (dummy median): {:.2f}".format(r2_score(ytest, ypred_dummy_median)))


Mean squared error (dummy): 0.04
Median absolute error (dummy): 0.13
r2_score (dummy mean): -0.00
r2_score (dummy median): -0.02


In [181]:
n =10

## MOR regressor with base SVR regressor

In [182]:
from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.utils import resample

# Create the SVR regressor
svr = SVR(epsilon=0.2)
#Create the Multioutput Regressor
model = MultiOutputRegressor(svr)
# Train the regressor
model = model.fit(xtrain, ytrain)
# Generate predictions for testing data
ypred = model.predict(xtest)


mse = []
r = []
mae = []
for i in range(n):
# Generate predictions for testing data
    # Resample
	resampled_xtest, resampled_ytest = resample(xtest, ytest, replace=True, n_samples=len(ytest), random_state=7+i)
	ypred = model.predict(resampled_xtest)
	mse.append(mean_squared_error(resampled_ytest,ypred))
	r.append(model.score(resampled_xtest, resampled_ytest))
	mae.append(mean_absolute_error(resampled_ytest,ypred))

print("Mean squared error (SVR): {:.2f} [{:.2f}, {:.2f}]".format(np.mean(mse), np.percentile(mse, 5), np.percentile(mse, 95)))
print("Mean abs error (SVR) {:.2f} [{:.2f}, {:.2f}]".format(np.mean(mae), np.percentile(mae, 5), np.percentile(mae, 95)))
print("R2 score (SVR): {:.2f} [{:.2f}, {:.2f}]".format(np.mean(r), np.percentile(r, 5), np.percentile(r, 95)))




Mean squared error (SVR): 0.04 [0.04, 0.05]
Mean abs error (SVR) 0.16 [0.16, 0.17]
R2 score (SVR): -0.05 [-0.09, -0.01]


In [183]:
max_depth = 30
# define model
model = RandomForestRegressor(n_estimators=100, max_depth=max_depth, random_state=7)
# fit model
model.fit(xtrain, ytrain)

mse = []
r = []
mae = []
for i in range(n):
# Generate predictions for testing data
    # Resample
	resampled_xtest, resampled_ytest = resample(xtest, ytest, replace=True, n_samples=len(ytest), random_state=7+i)
	ypred = model.predict(resampled_xtest)
	mse.append(mean_squared_error(resampled_ytest,ypred))
	r.append(model.score(resampled_xtest, resampled_ytest))
	mae.append(mean_absolute_error(resampled_ytest,ypred))

print("Mean squared error (RanFor): {:.2f} [{:.2f}, {:.2f}]".format(np.mean(mse), np.percentile(mse, 5), np.percentile(mse, 95)))
print("Mean abs error (RanFor) {:.2f} [{:.2f}, {:.2f}]".format(np.mean(mae), np.percentile(mae, 5), np.percentile(mae, 95)))
print("R2 score (RanFor): {:.2f} [{:.2f}, {:.2f}]".format(np.mean(r), np.percentile(r, 5), np.percentile(r, 95)))

									



Mean squared error (RanFor): 0.04 [0.04, 0.05]
Mean abs error (RanFor) 0.16 [0.15, 0.17]
R2 score (RanFor): -0.01 [-0.03, 0.00]


In [184]:
# define model
model = LinearRegression()
# fit model
model.fit(xtrain, ytrain)

mse = []
r = []
mae = []
for i in range(n):
# Generate predictions for testing data
    # Resample
	resampled_xtest, resampled_ytest = resample(xtest, ytest, replace=True, n_samples=len(ytest), random_state=7+i)
	ypred = model.predict(resampled_xtest)
	mse.append(mean_squared_error(resampled_ytest,ypred))
	r.append(model.score(resampled_xtest, resampled_ytest))
	mae.append(mean_absolute_error(resampled_ytest,ypred))

print("Mean squared error (LinReg): {:.2f} [{:.2f}, {:.2f}]".format(np.mean(mse), np.percentile(mse, 5), np.percentile(mse, 95)))
print("Mean abs error (LinReg) {:.2f} [{:.2f}, {:.2f}]".format(np.mean(mae), np.percentile(mae, 5), np.percentile(mae, 95)))
print("R2 score (LinReg): {:.2f} [{:.2f}, {:.2f}]".format(np.mean(r), np.percentile(r, 5), np.percentile(r, 95)))

#print("Mean squared error (LinReg): {:.2f}".format(mean_squared_error(ytest,
#																	ypred)))
#print("Mean abs error (LinReg): {:.2f}".format(mean_absolute_error(ytest,
#																	ypred)))																	
#print("R2 score (LinReg): {:.2f}".format(model.score(xtest, ytest)))		

Mean squared error (LinReg): 0.05 [0.04, 0.05]
Mean abs error (LinReg) 0.17 [0.16, 0.18]
R2 score (LinReg): -0.12 [-0.17, -0.08]


In [185]:
max_depth = 30
model = MultiOutputRegressor(
    RandomForestRegressor(n_estimators=100, max_depth=max_depth, random_state=7)
)
model.fit(xtrain, ytrain)

mse = []
r = []
mae = []
for i in range(n):
# Generate predictions for testing data
    # Resample
	resampled_xtest, resampled_ytest = resample(xtest, ytest, replace=True, n_samples=len(ytest), random_state=7+i)
	ypred = model.predict(resampled_xtest)
	mse.append(mean_squared_error(resampled_ytest,ypred))
	r.append(model.score(resampled_xtest, resampled_ytest))
	mae.append(mean_absolute_error(resampled_ytest,ypred))

print("Mean squared error (RanFor): {:.2f} [{:.2f}, {:.2f}]".format(np.mean(mse), np.percentile(mse, 5), np.percentile(mse, 95)))
print("Mean abs error (RanFor) {:.2f} [{:.2f}, {:.2f}]".format(np.mean(mae), np.percentile(mae, 5), np.percentile(mae, 95)))
print("R2 score (RanFor): {:.2f} [{:.2f}, {:.2f}]".format(np.mean(r), np.percentile(r, 5), np.percentile(r, 95)))

Mean squared error (RanFor): 0.04 [0.04, 0.05]
Mean abs error (RanFor) 0.16 [0.15, 0.17]
R2 score (RanFor): 0.00 [-0.01, 0.01]


In [186]:
in_dim = xtrain.shape[1]
out_dim = ytrain.shape[1]


model = Sequential()

model.add(Dense(5, input_dim=in_dim, kernel_initializer='he_uniform', activation='relu'))
model.add(Dense(4, input_dim=in_dim, kernel_initializer='he_uniform', activation='relu'))
model.add(Dense(3, input_dim=in_dim, kernel_initializer='he_uniform', activation='relu'))
model.add(Dense(2, input_dim=in_dim, kernel_initializer='he_uniform', activation='relu'))
model.add(Dense(1, input_dim=in_dim, kernel_initializer='he_uniform', activation='relu'))

model.add(Dense(out_dim))
model.compile(loss='mae', optimizer='adam')


model.fit(xtrain, ytrain, epochs=10, batch_size=2, validation_split = 0.2, verbose=1)
ypred = model.predict(xtest)

score = tf.keras.metrics.mean_absolute_error(
    ytest, ypred
)
score= np.array(score)
print(score.mean())

score_mse = tf.keras.metrics.mean_squared_error(
    ytest, ypred
)
score_mse =  np.array(score_mse)
print(score_mse.mean())

Epoch 1/10
322/322 [==============================] - 1s 2ms/step - loss: 0.3385 - val_loss: 0.2296
Epoch 2/10
322/322 [==============================] - 1s 2ms/step - loss: 0.1911 - val_loss: 0.1541
Epoch 3/10
322/322 [==============================] - 1s 2ms/step - loss: 0.1599 - val_loss: 0.1496
Epoch 4/10
322/322 [==============================] - 1s 2ms/step - loss: 0.1584 - val_loss: 0.1495
Epoch 5/10
322/322 [==============================] - 1s 2ms/step - loss: 0.1583 - val_loss: 0.1496
Epoch 6/10
322/322 [==============================] - 1s 2ms/step - loss: 0.1583 - val_loss: 0.1495
Epoch 7/10
322/322 [==============================] - 1s 2ms/step - loss: 0.1584 - val_loss: 0.1496
Epoch 8/10
322/322 [==============================] - 1s 2ms/step - loss: 0.1583 - val_loss: 0.1497
Epoch 9/10
322/322 [==============================] - 1s 2ms/step - loss: 0.1583 - val_loss: 0.1501
Epoch 10/10
322/322 [==============================] - 1s 2ms/step - loss: 0.1583 - val_loss: 0.1497

In [187]:
in_dim = xtrain.shape[1]
out_dim = ytrain.shape[1]


model = Sequential()

model.add(Dense(7, input_dim=in_dim, kernel_initializer='he_uniform', activation='relu'))


model.add(Dense(out_dim))
model.compile(loss='mae', optimizer='adam')


model.fit(xtrain, ytrain, epochs=10, batch_size=2, validation_split = 0.2, verbose=1)
ypred = model.predict(xtest)

score = tf.keras.metrics.mean_absolute_error(
    ytest, ypred
)
score= np.array(score)
print(score.mean())

score_mse = tf.keras.metrics.mean_squared_error(
    ytest, ypred
)
score_mse =  np.array(score_mse)
print(score_mse.mean())

Epoch 1/10
322/322 [==============================] - 0s 1ms/step - loss: 0.3874 - val_loss: 0.2542
Epoch 2/10
322/322 [==============================] - 1s 2ms/step - loss: 0.2126 - val_loss: 0.2099
Epoch 3/10
322/322 [==============================] - 0s 1ms/step - loss: 0.1816 - val_loss: 0.1932
Epoch 4/10
322/322 [==============================] - 0s 1ms/step - loss: 0.1688 - val_loss: 0.1834
Epoch 5/10
322/322 [==============================] - 0s 1ms/step - loss: 0.1625 - val_loss: 0.1800
Epoch 6/10
322/322 [==============================] - 0s 1ms/step - loss: 0.1583 - val_loss: 0.1791
Epoch 7/10
322/322 [==============================] - 0s 1ms/step - loss: 0.1558 - val_loss: 0.1779
Epoch 8/10
322/322 [==============================] - 0s 1ms/step - loss: 0.1539 - val_loss: 0.1797
Epoch 9/10
322/322 [==============================] - 0s 1ms/step - loss: 0.1519 - val_loss: 0.1783
Epoch 10/10
322/322 [==============================] - 0s 1ms/step - loss: 0.1501 - val_loss: 0.1795